In [2]:
import pandas as pd
from pathlib import Path
from config import geoapify_key
import requests
import json
from pprint import pprint
import re

In [3]:
# files  = [
#     "listings_toronto_type1_pages001-162_2024-01-30.csv",
#     "listings_toronto_type2_pages001-016_2024-01-30.csv",
#     "listings_toronto_type3_pages001-015_2024-01-30.csv",
#     "listings_toronto_type5_pages001-050_2024-01-30.csv"
# ]

files = ["listings_with_locations.csv"]

li = []

for file in files:
    file_path = Path("data/" + file)
    partial_df = pd.read_csv(file_path, index_col="mls_id", header=0)
    li.append(partial_df)

listings_df = pd.concat(li, axis=0)

listings_df.head()

,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude
mls_id,,,,,,,,,,,,,
E8018446,https://toronto.listing.ca/286-main-st-911.E80...,286 Main St 911,619900,1,1,1,Main St,East End-Danforth,Toronto,condo_apartment,2024-01-30,43.687469,-79.301861
C7266728,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,529000,1,1,0,Queen St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.650343,-79.387806
W7239426,https://toronto.listing.ca/10-park-lawn-rd-140...,10 Park Lawn Rd 1408,624900,1,1,1,Park Lawn Rd,Mimico,Toronto,condo_apartment,2024-01-30,43.622946,-79.481658
E8030950,https://toronto.listing.ca/665-queen-st-402.E8...,665 Queen St 402,899900,2,2,0,Queen St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.658361,-79.351159
E8030860,https://toronto.listing.ca/1190-dundas-st-925....,1190 Dundas St 925,599900,1,1,0,Dundas St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.665267,-79.341034


In [4]:

# Set the search parameters
target_city = "Toronto, Canada"

# Build URL using the geocode endpoint
geo_url = f"https://api.geoapify.com/v1/geocode/search?city=Toronto&type=city&format=json&apiKey={geoapify_key}"
geo_response = requests.get(geo_url).json()
#pprint(geo_response)
toronto_place_id = geo_response['results'][0]['place_id']
toronto_place_id

'512c13d96292d853c0596a04d149a5d34540f00101f90173f2040000000000c00208'

In [5]:
if 'latitude' not in listings_df.columns:
    listings_df['latitude'] = ""
if 'longitude' not in listings_df.columns:
    listings_df['longitude'] = ""
failed_addresses = []
address_count = 0

for i, row in listings_df.iterrows():
    address_count += 1
    address = listings_df.loc[i, 'address']
    original_address = address

    if pd.isnull(listings_df.loc[i, 'latitude']) or listings_df.loc[i, 'latitude']=="":
    
        #Clean up Address
        address = address.replace("#", "")
        address = address.replace("/", "")
        address = address.replace("-", "")
        address = address.rstrip("0123456789 ")
        address = address.removesuffix(' Ph')
        address = address.removesuffix(' Tp')
        address = address.removesuffix(' Lph')
        address = address.removesuffix(' Sph')
        address = address.removesuffix(' Uph')
        address = address.removesuffix(' N/A')
        address = address.removesuffix(' NA')
        address = address.removesuffix(' Nt')
        address = address.rstrip("0123456789 ")
        address = address.replace(" Don Way", " DonWay")
        address = address.replace("Queensway Way", "Queensway")
        address = address.replace("Gfwy", "Golfway")
        address = address.replace("Glwy", "Glenway")
        if address[-3:] == " Sq":
            address = address[:-3] + " Square"
        if address[-3:] == " Dr":
            address = address[:-3] + " Drive"
        if address[-2:] == " W":
            address = address[:-2] + " West"
        if address[-2:] == " E":
            address = address[:-2] + " East"
        
        pattern_1 = re.compile(r'\s\d+[A-Za-z]$')
        match = pattern_1.search(address)
        if match:
            address = address.replace(match.group(0), '')
        
        pattern_2 = re.compile(r'\s\[A-Za-z]$')
        match2 = pattern_2.search(address)
        if match2:
            address = address.replace(match2.group(0), '')
        
        address_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={address}&filter=place:{toronto_place_id}&format=json&apiKey={geoapify_key}"
        address_response = requests.get(address_url).json()
        try:
            listings_df.at[i, 'latitude'] = address_response['results'][0]['lat']
            listings_df.at[i, 'longitude'] = address_response['results'][0]['lon']
            print(f'API call {address_count} successful for {address}')
        except IndexError:
            print(f'API call {address_count} failed for {address}')
            #If first attempt fails remove the house number and check for the street address
            address = address.lstrip("0123456789 ")
            address_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={address}&filter=place:{toronto_place_id}&format=json&apiKey={geoapify_key}"
            address_response = requests.get(address_url).json()
            try:
                listings_df.at[i, 'latitude'] = address_response['results'][0]['lat']
                listings_df.at[i, 'longitude'] = address_response['results'][0]['lon']
                print(f'API call {address_count} successful for {address}')
            except:
                #append the failed addresses to a list to keep track of them
                failed_addresses.append(original_address)
                print(f'API call {address_count} failed for {address}')

API call 69 failed for 33 Frederick Todd Way
API call 69 failed for Frederick Todd Way
API call 119 failed for 33 Frederick Todd Way
API call 119 failed for Frederick Todd Way
API call 211 failed for 52 Forest Manor Rd P
API call 211 failed for Forest Manor Rd P
API call 419 failed for 55 Merchants Wharf St
API call 419 failed for Merchants Wharf St
API call 489 failed for 33 Frederick Todd Way
API call 489 failed for Frederick Todd Way
API call 726 failed for 27 Rean Ave
API call 726 failed for Rean Ave


KeyboardInterrupt: 

In [ ]:

with open("data/failed_addresses.txt", "w") as output:
    for entry in failed_addresses:
        output.write(str(entry))
        output.write('\n')


In [ ]:
listings_df.head()

,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude
mls_id,,,,,,,,,,,,,
E8018446,https://toronto.listing.ca/286-main-st-911.E80...,286 Main St 911,619900,1,1,1,Main St,East End-Danforth,Toronto,condo_apartment,2024-01-30,43.687469,-79.301861
C7266728,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,529000,1,1,0,Queen St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.650343,-79.387806
W7239426,https://toronto.listing.ca/10-park-lawn-rd-140...,10 Park Lawn Rd 1408,624900,1,1,1,Park Lawn Rd,Mimico,Toronto,condo_apartment,2024-01-30,43.622946,-79.481658
E8030950,https://toronto.listing.ca/665-queen-st-402.E8...,665 Queen St 402,899900,2,2,0,Queen St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.658361,-79.351159
E8030860,https://toronto.listing.ca/1190-dundas-st-925....,1190 Dundas St 925,599900,1,1,0,Dundas St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.665267,-79.341034


In [ ]:
listings_df.to_csv('data/listings_with_locations.csv')